<h1 align=center>Capítulo 3</h1>
<h2 align=center>Número, Datas e Horas</h2>
<p align=center><img src=https://www.korntraducoes.com.br/wp-content/uploads/2019/07/Numeros_horarios_datas_endere%C3%A7os_tradu%C3%A7%C3%B5es_korn_traducoes.jpg width=500></p>

Realizar cálculos matemáticos com números inteiros e de ponto flutuante é fácil em Python. No entanto, se você precisar realizar cálculos com frações, matrizes ou datas e horas, será necessário um pouco mais de trabalho. O foco deste capítulo são esses tópicos.

## 3.1. Arredondando Valores Numéricos

**Problema**
Você deseja arredondar um número de ponto flutuante para um número fixo de casas decimais.

**Solução**
Para arredondamento simples, use a função `round(value, ndigits)` embutida. Por exemplo:

In [1]:
round(1.23, 1)

1.2

In [2]:
round(1.27, 1)

1.3

In [3]:
round(-1.27, 1)

-1.3

In [4]:
round(1.25361,3)

1.254

Quando um valor está exatamente na metade entre duas opções, o comportamento de arredondar é arredondar para o dígito par mais próximo. Ou seja, valores como 1.5 ou 2.5 são arredondados para 2.

O número de dígitos dado a `round()` pode ser negativo, caso em que o arredondamento ocorre para dezenas, centenas, milhares e assim por diante. Por exemplo:

In [5]:
a = 1627731
round(a, -1)

1627730

In [6]:
round(a, -2)

1627700

In [7]:
round(a, -3)

1628000

**Discussão**

Não confunda arredondamento com formatação de um valor para saída. Se seu objetivo é simplesmente produzir um valor numérico com um certo número de casas decimais, você normalmente não precisa usar `round()`. Em vez disso, basta especificar a precisão desejada ao formatar. Por exemplo:

In [8]:
x = 1.23456
format(x, '0.2f')

'1.23'

In [9]:
format(x, '0.3f')

'1.235'

In [10]:
'value is {:0.3f}'.format(x)

'value is 1.235'

Além disso, resista ao desejo de arredondar números de ponto flutuante para “corrigir” problemas de precisão percebidos. Por exemplo, você pode estar inclinado a fazer isso:

In [11]:
a = 2.1
b = 4.2
c = a + b
c

6.300000000000001

In [12]:
c = round(c, 2) # "Fix" result (???)
c

6.3

Para a maioria dos aplicativos que envolvem ponto flutuante, simplesmente não é necessário (ou recomendado) fazer isso. Embora existam pequenos erros introduzidos nos cálculos, o comportamento desses erros é compreendido e tolerado. Se evitar tais erros for importante (por exemplo, em aplicações financeiras, talvez), considere o uso do módulo `decimal`, que é discutido na próxima receita.

## 3.2. Executando Cálculos Decimais Precisos

**Problema**

Você precisa realizar cálculos precisos com números decimais e não quer os pequenos erros que ocorrem naturalmente com floats.

**Solução**

Um problema bem conhecido com números de ponto flutuante é que eles não podem representar com precisão todos os decimais de base 10. Além disso, mesmo cálculos matemáticos simples introduzem pequenos erros. Por exemplo:

In [13]:
a = 4.2
b = 2.1
a + b

6.300000000000001

In [14]:
(a + b) == 6.3

False

Esses erros são um “recurso” da CPU subjacente e da aritmética IEEE 754 realizada por sua unidade de ponto flutuante. Como o tipo de dados float do Python armazena dados usando a representação nativa, não há nada que você possa fazer para evitar esses erros se escrever seu código usando instâncias float.

Se você deseja mais precisão (e está disposto a desistir de algum desempenho), pode usar o módulo `decimal`:

In [15]:
from decimal import Decimal
a = Decimal('4.2')
b = Decimal('2.1')
a + b

Decimal('6.3')

In [16]:
print(a + b)

6.3


In [17]:
(a + b) == Decimal('6.3')

True

À primeira vista, pode parecer um pouco estranho (ou seja, especificar números como strings). No entanto, os objetos `Decimal` funcionam de todas as maneiras que você esperaria (suportando todas as operações matemáticas usuais, etc.). Se você imprimi-los ou usá-los em funções de formatação de strings, eles se parecerão com números normais.

Uma característica importante do `decimal` é que ele permite controlar diferentes aspectos dos cálculos, incluindo número de dígitos e arredondamento. Para fazer isso, você cria um contexto local e altera suas configurações. Por exemplo:

In [18]:
from decimal import localcontext
a = Decimal('1.3')
b = Decimal('1.7')
print(a / b)

0.7647058823529411764705882353


In [19]:
with localcontext() as ctx:
    ctx.prec = 3
    print(a / b)

0.765


In [20]:
with localcontext() as ctx:
    ctx.prec = 50
    print(a / b)

0.76470588235294117647058823529411764705882352941176


**Discussão**

O módulo `decimal` implementa a “Especificação Aritmética Decimal Geral” da IBM. Desnecessário dizer que há um grande número de opções de configuração que estão além do escopo deste livro.

Os recém-chegados ao Python podem estar inclinados a usar o módulo `decimal` para solucionar problemas de precisão percebidos com o tipo de dados `float`. No entanto, é muito importante entender o domínio do seu aplicativo. Se você estiver trabalhando com problemas de ciência ou engenharia, computação gráfica ou a maioria das coisas de natureza científica, é simplesmente mais comum usar o tipo normal de ponto flutuante. Por um lado, muito poucas coisas no mundo real são medidas com os 17 dígitos de precisão que os flutuadores fornecem. Assim, pequenos erros introduzidos nos cálculos simplesmente não importam. Em segundo lugar, o desempenho de floats nativos é significativamente mais rápido - algo importante se você estiver realizando um grande número de cálculos.

Dito isso, você não pode ignorar os erros completamente. Os matemáticos passaram muito tempo estudando vários algoritmos, e alguns lidam com erros melhor do que outros. Você também precisa ter um pouco de cuidado com os efeitos devido a coisas como cancelamento subtrativo e adição de números grandes e pequenos. Por exemplo:

In [21]:
nums = [1.23e+18, 1, -1.23e+18]
sum(nums) # Notice how 1 disappears

0.0

Este último exemplo pode ser resolvido usando uma implementação mais precisa em `math.fsum()`:

In [22]:
import math
math.fsum(nums)

1.0

No entanto, para outros algoritmos, você realmente precisa estudar o algoritmo e entender suas propriedades de propagação de erro.

Tudo isso dito, o principal uso do módulo decimal é em programas que envolvem coisas como finanças. Nesses programas, é extremamente irritante ter pequenos erros no cálculo. Assim, decimal fornece uma maneira de evitar isso. Também é comum encontrar objetos Decimal quando o Python faz interface com bancos de dados – novamente, especialmente ao acessar dados financeiros.

## 3.3. Formatando Números para Saída

**Problema**

Você precisa formatar um número para saída, controlando o número de dígitos, alinhamento, inclusão de um separador de milhares e outros detalhes.

**Solução**

Para formatar um único número para saída, use a função `format()` integrada. Por exemplo:

In [23]:
x = 1234.56789
# Dois decimais
format(x, '0.2f')

'1234.57'

In [24]:
# Justificado Direita 10 caracteres
format(x, '>10.1f')

'    1234.6'

In [25]:
# Esquerda justificado 
format(x, '<10.1f')

'1234.6    '

In [26]:
# Centralizado
format(x, '^10.1f')

'  1234.6  '

In [27]:
# Inclusão de milhar
format(x, ',')

'1,234.56789'

In [28]:
>>> format(x, '0,.1f')

'1,234.6'

Se você quiser usar a notação exponencial, altere o `f` para um `e` ou `E`, dependendo do caso que deseja usar para o especificador exponencial. Por exemplo:

In [29]:
format(x, 'e')

'1.234568e+03'

In [30]:
format(x, '0.2E')

'1.23E+03'

A forma geral de largura e precisão em ambos os casos é **'\[<>^\]?width\[,\]?(.digits)?'** onde largura e dígitos são inteiros e `?` significa peças opcionais. Os mesmos códigos de formato também são usados no método `.format()` de strings. Por exemplo:

In [31]:
'The value is {:0,.2f}'.format(x)

'The value is 1,234.57'

**Discussão**

A formatação de números para saída geralmente é simples. A técnica mostrada funciona tanto para números de ponto flutuante quanto para números decimais no módulo decimal. Quando o número de dígitos é restrito, os valores são arredondados de acordo com as mesmas regras da função `round()`. Por exemplo:

In [32]:
x

1234.56789

In [33]:
format(x, '0.1f')

'1234.6'

In [34]:
format(-x, '0.1f')

'-1234.6'

A formatação de valores com um separador de milhares não reconhece a localidade. Se você precisar levar isso em consideração, poderá investigar funções no módulo `locale`. Você também pode trocar caracteres separadores usando o método `translate()` de strings. Por exemplo:

In [35]:
swap_separators = { ord('.'):',', ord(','):'.' }
format(x, ',').translate(swap_separators)

'1.234,56789'

Em muitos códigos Python, os números são formatados usando o operador `%`. Por exemplo:

In [36]:
'%0.2f' % x

'1234.57'

In [37]:
'%10.1f' % x

'    1234.6'

In [38]:
'%-10.1f' % x

'1234.6    '

Essa formatação ainda é aceitável, mas menos poderosa que o método `format()` mais moderno. Por exemplo, alguns recursos (por exemplo, adicionar separadores de milhares) não são suportados ao usar o operador `%` para formatar números.

## 3.4. Trabalhando com números inteiros binários, octais e hexadecimais

**Problema**

Você precisa converter ou gerar números inteiros representados por dígitos binários, octais ou hexadecimais.

**Solução**

Para converter um inteiro em uma string de texto binária, octal ou hexadecimal, use as funções `bin()`, `oct()` ou `hex()`, respectivamente:

In [39]:
x = 1234

In [40]:
bin(x)

'0b10011010010'

In [41]:
oct(x)

'0o2322'

In [42]:
hex(x)

'0x4d2'

Como alternativa, você pode usar a função `format()` se não quiser que os prefixos 0b, 0o ou 0x apareçam. Por exemplo:

In [43]:
format(x, 'b')

'10011010010'

In [44]:
format(x, 'o')

'2322'

In [45]:
format(x, 'x')

'4d2'

Os inteiros são assinados, portanto, se você estiver trabalhando com números negativos, a saída também incluirá um sinal. Por exemplo:

In [46]:
x = -1234
format(x, 'b')

'-10011010010'

In [47]:
format(x, 'x')

'-4d2'

Se você precisar produzir um valor sem sinal, precisará adicionar o valor máximo para definir o comprimento do bit. Por exemplo, para mostrar um valor de 32 bits, use o seguinte:

In [48]:
x = -1234
format(2**32 + x, 'b')

'11111111111111111111101100101110'

In [49]:
format(2**32 + x, 'x')

'fffffb2e'

Para converter strings inteiras em bases diferentes, basta usar a função `int()` com uma base apropriada. Por exemplo:

In [50]:
int('4d2', 16)

1234

In [51]:
int('10011010010', 2)

1234

**Discussão**

Na maioria das vezes, trabalhar com inteiros binários, octais e hexadecimais é simples. Apenas lembre-se de que essas conversões dizem respeito apenas à conversão de inteiros de e para uma representação textual. Nos bastidores, há apenas um tipo inteiro. Finalmente, há um cuidado para os programadores que usam octal. A sintaxe do Python para especificar valores octais é um pouco diferente de muitas outras linguagens. Por exemplo, se você tentar algo assim, receberá um erro de sintaxe:
~~~python
import os
os.chmod('script.py', 0755)
    File "<stdin>", line 1
      os.chmod('script.py', 0755)
 SyntaxError: invalid token                            ^
~~~
Certifique-se de prefixar o valor octal com `0o`, conforme mostrado aqui:

In [52]:
import os
os.chmod('somefile.txt', 0o755)

## 3.5. Empacotando e desempacotando grandes números inteiros de bytes

**Problema**

Você tem uma string de bytes e precisa descompactá-la em um valor inteiro. Como alternativa, você precisa converter um número inteiro grande de volta em uma string de bytes.

**Solução**

Suponha que seu programa precise trabalhar com uma cadeia de bytes de 16 elementos que contenha um valor inteiro de 128 bits. Por exemplo:

In [53]:
data = b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

Para interpretar os bytes como um inteiro, use `int.from_bytes()` e especifique a ordem dos bytes assim:

In [54]:
len(data)

16

In [55]:
int.from_bytes(data, 'little')

69120565665751139577663547927094891008

In [56]:
int.from_bytes(data,'big')

94522842520747284487117727783387188

Para converter um valor inteiro grande de volta em uma string de bytes, use o método `int.to_bytes()`, especificando o número de bytes e a ordem dos bytes. Por exemplo:

In [57]:
x = 94522842520747284487117727783387188
x.to_bytes(16,'big')

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [58]:
x.to_bytes(16,'little')

b'4\x00#\x00\x01\xef\xcd\x00\xab\x90x\x00V4\x12\x00'

**Discussão**

Converter valores inteiros grandes de e para strings de bytes não é uma operação comum. No entanto, às vezes surge em certos domínios de aplicativos, como criptografia ou rede. Por exemplo, os endereços de rede IPv6 são representados como inteiros de 128 bits. Se você estiver escrevendo um código que precisa extrair esses valores de um registro de dados, poderá enfrentar esse problema.

Como alternativa a esta receita, você pode estar inclinado a descompactar valores usando o módulo `struct`, conforme descrito na Receita 6.11. Isso funciona, mas o tamanho dos inteiros que podem ser descompactados com `struct` é limitado. Assim, você precisaria descompactar vários valores e combiná-los para criar o valor final. Por exemplo:

In [59]:
data

b'\x00\x124V\x00x\x90\xab\x00\xcd\xef\x01\x00#\x004'

In [60]:
import struct
hi, lo = struct.unpack('>QQ',data)

In [61]:
(hi<<64) + lo

94522842520747284487117727783387188

A especificação da ordem dos bytes (**little** ou **big**) apenas indica se os bytes que compõem o valor inteiro estão listados do menos para o mais significativo ou vice-versa. Isso é fácil de visualizar usando um valor hexadecimal cuidadosamente elaborado:

In [62]:
x = 0x01020304
x.to_bytes(4, 'big')

b'\x01\x02\x03\x04'

In [63]:
x.to_bytes(4, 'little')

b'\x04\x03\x02\x01'

Se você tentar empacotar um inteiro em uma string de bytes, mas não caber, você receberá um erro. Você pode usar o método `int.bit_length()` para determinar quantos bits são necessários para armazenar um valor, se necessário:

In [64]:
x = 523 ** 23
x

335381300113661875107536852714019056160355655333978849017944067

~~~python
x.to_bytes(16, 'little')
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
 OverflowError: int too big to convert
~~~

In [65]:
x.bit_length()

208

In [66]:
nbytes, rem = divmod(x.bit_length(), 8)
if rem:
    nbytes += 1

x.to_bytes(nbytes, 'little')

b'\x03X\xf1\x82iT\x96\xac\xc7c\x16\xf3\xb9\xcf\x18\xee\xec\x91\xd1\x98\xa2\xc8\xd9R\xb5\xd0'

## 3.6. Executando matemática de valor complexo

**Problema**

Seu código para interagir com o esquema de autenticação da Web mais recente encontrou uma singularidade e sua única solução é contorná-lo no plano complexo. Ou talvez você só precise realizar alguns cálculos usando números complexos.

**Solução**

Números complexos podem ser especificados usando a função `complex(real, imag)` ou por números de ponto flutuante com um sufixo j. Por exemplo:

In [67]:
a = complex(2, 4)
b = 3 - 5j
a

(2+4j)

In [68]:
b

(3-5j)

Os valores reais, imaginários e conjugados são fáceis de obter, como mostrado aqui:

In [69]:
a.real

2.0

In [70]:
a.imag

4.0

In [71]:
a.conjugate()

(2-4j)

Além disso, todos os operadores matemáticos usuais funcionam:

In [72]:
a + b

(5-1j)

In [73]:
a * b

(26+2j)

In [74]:
a / b

(-0.4117647058823529+0.6470588235294118j)

In [75]:
abs(a)

4.47213595499958

Para executar funções adicionais de valor complexo, como senos, cossenos ou raízes quadradas, use o módulo `cmath`:

In [76]:
import cmath
cmath.sin(a)

(24.83130584894638-11.356612711218174j)

In [77]:
cmath.cos(a)

(-11.36423470640106-24.814651485634187j)

In [78]:
cmath.exp(a)

(-4.829809383269385-5.5920560936409816j)

**Discussão**

A maioria dos módulos relacionados à matemática do Python estão cientes de valores complexos. Por exemplo, se você usar `numpy`, é simples criar arrays de valores complexos e realizar operações neles:

In [79]:
import numpy as np
a = np.array([2+3j, 4+5j, 6-7j, 8+9j])
a

array([2.+3.j, 4.+5.j, 6.-7.j, 8.+9.j])

In [80]:
a + 2

array([ 4.+3.j,  6.+5.j,  8.-7.j, 10.+9.j])

In [81]:
np.sin(a)

array([   9.15449915  -4.16890696j,  -56.16227422 -48.50245524j,
       -153.20827755-526.47684926j, 4008.42651446-589.49948373j])

As funções matemáticas padrão do Python não produzem valores complexos por padrão, portanto, é improvável que tal valor apareça acidentalmente em seu código. Por exemplo:
~~~python
import math
math.sqrt(-1)
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
 ValueError: math domain error
~~~

Se você deseja que números complexos sejam produzidos como resultado, você deve usar `cmath` explicitamente ou declarar o uso de um tipo complexo em bibliotecas que os conhecem. Por exemplo:

In [82]:
import cmath
cmath.sqrt(-1)

1j

## 3.7. Trabalhando com Infinity e NaNs

**Problema**

Você precisa criar ou testar os valores de ponto flutuante de infinito, infinito negativo ou NaN (não um número).

**Solução**

Python não tem sintaxe especial para representar esses valores especiais de ponto flutuante, mas eles podem ser criados usando `float()`. Por exemplo:

In [83]:
a = float('inf')
b = float('-inf')
c = float('nan')
a

inf

In [84]:
b

-inf

In [85]:
c

nan

Para testar a presença desses valores, use as funções `math.isinf()` e `math.isnan()`. Por exemplo:

In [86]:
math.isinf(a)

True

In [87]:
math.isnan(c)

True

**Discussão**

Para obter informações mais detalhadas sobre esses valores especiais de ponto flutuante, consulte a especificação IEEE 754. No entanto, existem alguns detalhes complicados a serem observados, especialmente relacionados a comparações e operadores.

Valores infinitos se propagarão em cálculos de maneira matemática. Por exemplo:

In [88]:
a = float('inf')
a + 45

inf

In [89]:
a * 10

inf

In [90]:
10 / a

0.0

No entanto, certas operações são indefinidas e resultarão em um resultado `NaN`. Por exemplo:

In [91]:
a = float('inf')
a/a

nan

In [92]:
b = float('-inf')
a + b

nan

Os valores **NaN** se propagam por todas as operações sem gerar uma exceção. Por exemplo:

In [93]:
c = float('nan')
c + 23

nan

In [94]:
c / 2

nan

In [95]:
math.sqrt(c)

nan

Uma característica sutil dos valores NaN é que eles nunca se comparam como iguais. Por exemplo:

In [96]:
c = float('nan')
d = float('nan')
c == d

False

In [97]:
c is d

False

Por causa disso, a única maneira segura de testar um valor `NaN` é usar `math.isnan()`, conforme mostrado nesta receita.

Às vezes, os programadores desejam alterar o comportamento do Python para gerar exceções quando as operações resultam em um resultado infinito ou NaN. O módulo `fpectl` pode ser usado para ajustar esse comportamento, mas não está habilitado em uma compilação padrão do Python, é dependente da plataforma e realmente destinado apenas a programadores de nível especialista. Consulte a documentação on-line do Python para obter mais detalhes.

## 3.8. Calculando com frações

**Problema**

Você entrou em uma máquina do tempo e de repente se viu trabalhando em problemas de lição de casa de nível elementar envolvendo frações. Ou talvez você esteja escrevendo um código para fazer cálculos envolvendo medições feitas em sua marcenaria.

**Solução**

O módulo de frações pode ser usado para realizar cálculos matemáticos envolvendo frações. Por exemplo:

In [98]:
from fractions import Fraction
a = Fraction(5, 4)
b = Fraction(7, 16)
print(a + b)

27/16


In [99]:
print(a * b)

35/64


In [100]:
# Getting numerator/denominator
c = a * b
c.numerator

35

In [101]:
c.denominator

64

In [102]:
# Converting to a float
float(c)

0.546875

In [103]:
# Limiting the denominator of a value
print(c.limit_denominator(8))

4/7


In [104]:
# Converting a float to a fraction
x = 3.75
y = Fraction(*x.as_integer_ratio())
y

Fraction(15, 4)

**Discussão**
Calcular com frações não ocorre com frequência na maioria dos programas, mas há situações em que pode fazer sentido usá-las. Por exemplo, permitir que um programa aceite unidades de medida em frações e realizar cálculos com elas nesse formato pode aliviar a necessidade de um usuário fazer conversões manualmente para decimais ou floats.

## 3.9. Calculando com grandes matrizes numéricas

**Problema**

Você precisa realizar cálculos em grandes conjuntos de dados numéricos, como matrizes ou grades.

**Solução**

Para qualquer computação pesada envolvendo arrays, use a biblioteca `NumPy`. O principal recurso do `NumPy` é que ele fornece ao Python um objeto de matriz que é muito mais eficiente e mais adequado para cálculos matemáticos do que uma lista padrão do Python. Aqui está um pequeno exemplo que ilustra diferenças comportamentais importantes entre listas e matrizes `NumPy`:

In [105]:
# Python lists
x = [1, 2, 3, 4]
y = [5, 6, 7, 8]
x * 2

[1, 2, 3, 4, 1, 2, 3, 4]

~~~python
x + 10
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
 TypeError: can only concatenate list (not "int") to list
~~~

In [106]:
x + y

[1, 2, 3, 4, 5, 6, 7, 8]

In [107]:
# Numpy arrays
import numpy as np
ax = np.array([1, 2, 3, 4])
ay = np.array([5, 6, 7, 8])
ax * 2

array([2, 4, 6, 8])

In [108]:
ax + 10

array([11, 12, 13, 14])

In [109]:
ax + ay

array([ 6,  8, 10, 12])

In [110]:
ax * ay

array([ 5, 12, 21, 32])

Como você pode ver, as operações matemáticas básicas envolvendo arrays se comportam de maneira diferente.

Especificamente, as operações escalares (por exemplo, `ax * 2` ou `ax + 10`) aplicam a operação elemento a elemento. Além disso, a execução de operações matemáticas quando ambos os operandos são matrizes aplica a operação a todos os elementos e produz uma nova matriz.

O fato de que as operações matemáticas se aplicam a todos os elementos simultaneamente torna muito fácil e rápido calcular funções em uma matriz inteira. Por exemplo, se você quiser calcular o valor de um polinômio:

In [111]:
def f(x):
    return 3*x**2 - 2*x + 7

In [112]:
f(ax)

array([ 8, 15, 28, 47])

O `NumPy` fornece uma coleção de “funções universais” que também permitem operações de matriz. Estas são substituições para funções similares normalmente encontradas no módulo matemático. Por exemplo:

In [113]:
np.sqrt(ax)

array([1.        , 1.41421356, 1.73205081, 2.        ])

In [114]:
np.cos(ax)

array([ 0.54030231, -0.41614684, -0.9899925 , -0.65364362])

O uso de funções universais pode ser centenas de vezes mais rápido do que fazer um loop sobre os elementos da matriz um de cada vez e realizar cálculos usando funções no módulo `math`. Assim, você deve preferir seu uso sempre que possível.

Nos bastidores, os arrays `NumPy` são alocados da mesma maneira que em C ou Fortran. Ou seja, são regiões de memória grandes e contíguas que consistem em um tipo de dados homogêneo. Por causa disso, é possível fazer arrays muito maiores do que qualquer coisa que você normalmente colocaria em uma lista Python. Por exemplo, se você quiser fazer uma grade bidimensional de 10.000 por 10.000 floats, isso não é um problema:

In [115]:
grid = np.zeros(shape=(10000,10000), dtype=float)
grid

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Todas as operações usuais ainda se aplicam a todos os elementos simultaneamente:

In [116]:
grid += 10
grid

array([[10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       ...,
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.],
       [10., 10., 10., ..., 10., 10., 10.]])

In [117]:
np.sin(grid)

array([[-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111],
       [-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111],
       [-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111],
       ...,
       [-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111],
       [-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111],
       [-0.54402111, -0.54402111, -0.54402111, ..., -0.54402111,
        -0.54402111, -0.54402111]])

Um aspecto extremamente notável do NumPy é a maneira como ele estende a funcionalidade de indexação de lista do Python - especialmente com matrizes multidimensionais. Para ilustrar, faça um array bidimensional simples e tente alguns experimentos:

In [118]:
a = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
a

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [119]:
# Select row 1
a[1]

array([5, 6, 7, 8])

In [120]:
# Select column 1
a[:,1]

array([ 2,  6, 10])

In [121]:
# Select a subregion and change it
a[1:3, 1:3]

array([[ 6,  7],
       [10, 11]])

In [122]:
a[1:3, 1:3] += 10
a

array([[ 1,  2,  3,  4],
       [ 5, 16, 17,  8],
       [ 9, 20, 21, 12]])

In [123]:
# Broadcast a row vector across an operation on all rows
a + [100, 101, 102, 103]

array([[101, 103, 105, 107],
       [105, 117, 119, 111],
       [109, 121, 123, 115]])

In [124]:
a

array([[ 1,  2,  3,  4],
       [ 5, 16, 17,  8],
       [ 9, 20, 21, 12]])

In [125]:
# Conditional assignment on an array
np.where(a < 10, a, 10)

array([[ 1,  2,  3,  4],
       [ 5, 10, 10,  8],
       [ 9, 10, 10, 10]])

**Discussão**

O NumPy é a base para um grande número de bibliotecas de ciência e engenharia em Python. É também um dos módulos maiores e mais complicados em uso generalizado.
Dito isso, ainda é possível realizar coisas úteis com o NumPy começando com exemplos simples e brincando.
Uma observação sobre o uso é que é relativamente comum usar a instrução import numpy como np, conforme mostrado na solução. Isso simplesmente encurta o nome para algo que é mais conveniente para digitar repetidamente em seu programa.
Para obter mais informações, você definitivamente precisa visitar http://www.numpy.org.

## 3.10. Executando cálculos de matriz e álgebra linear

**Problema**

Você precisa realizar operações de matriz e álgebra linear, como multiplicação de matrizes, encontrar determinantes, resolver equações lineares e assim por diante.

**Solução**

A biblioteca NumPy possui um objeto de matriz que pode ser usado para essa finalidade. As matrizes são um pouco semelhantes aos objetos de matriz descritos na Receita 3.9, mas seguem regras de álgebra linear para computação. Aqui está um exemplo que ilustra alguns recursos essenciais:

In [126]:
import numpy as np
m = np.matrix([[1,-2,3],[0,4,5],[7,8,-9]])
m

matrix([[ 1, -2,  3],
        [ 0,  4,  5],
        [ 7,  8, -9]])

In [127]:
# Return transpose
m.T

matrix([[ 1,  0,  7],
        [-2,  4,  8],
        [ 3,  5, -9]])

In [128]:
# Return inverse
m.I

matrix([[ 0.33043478, -0.02608696,  0.09565217],
        [-0.15217391,  0.13043478,  0.02173913],
        [ 0.12173913,  0.09565217, -0.0173913 ]])

In [129]:
# Create a vector and multiply
v = np.matrix([[2],[3],[4]])
v

matrix([[2],
        [3],
        [4]])

In [130]:
m * v

matrix([[ 8],
        [32],
        [ 2]])

Mais operações podem ser encontradas no subpacote `numpy.linalg`. Por exemplo:

In [131]:
import numpy.linalg

# Determinant
numpy.linalg.det(m)

-229.99999999999983

In [132]:
# Eigenvalues
numpy.linalg.eigvals(m)

array([-13.11474312,   2.75956154,   6.35518158])

In [133]:
# Solve for x in mx = v
x = numpy.linalg.solve(m, v)
x

matrix([[0.96521739],
        [0.17391304],
        [0.46086957]])

In [134]:
m * x

matrix([[2.],
        [3.],
        [4.]])

In [135]:
v

matrix([[2],
        [3],
        [4]])

**Discussão**

A álgebra linear é obviamente um tópico enorme que está muito além do escopo deste livro de receitas. No entanto, se você precisar manipular matrizes e vetores, o NumPy é um bom ponto de partida. Visite http://www.numpy.org para obter informações mais detalhadas.

## 3.11. Escolhendo coisas ao acaso

**Problema**

Você deseja escolher itens aleatórios de uma sequência ou gerar números aleatórios.

**Solução**

O módulo aleatório tem várias funções para números aleatórios e escolha de itens aleatórios. Por exemplo, para escolher um item aleatório de uma sequência, use `random.choice()`:

In [136]:
import random
values = [1, 2, 3, 4, 5, 6]
random.choice(values)

6

In [137]:
random.choice(values)

6

In [138]:
random.choice(values)

4

Para obter uma amostra de N itens em que os itens selecionados são removidos de consideração adicional, use `random.sample()`:

In [139]:
random.sample(values, 2)

[4, 1]

In [140]:
random.sample(values, 2)

[5, 6]

In [141]:
random.sample(values, 3)

[5, 2, 1]

Se você simplesmente deseja embaralhar itens em uma sequência no local, use `random.shuffle()`:

In [142]:
random.shuffle(values)
values

[6, 2, 4, 1, 3, 5]

In [143]:
random.shuffle(values)
values

[5, 3, 2, 4, 1, 6]

Para produzir números inteiros aleatórios, use  `random.randint()´:

In [144]:
random.randint(0,10)

7

In [145]:
random.randint(0,10)

1

In [146]:
random.randint(0,10)

1

Para produzir valores de ponto flutuante uniformes no intervalo de 0 a 1, use `random.random()`:

In [147]:
random.random()

0.35460875074004705

In [148]:
random.random()

0.6896370029982141

Para obter N bits aleatórios expressos como um inteiro, use `random.getrandbits()`:

In [149]:
random.getrandbits(200)

566910195002179899826530369386363487786647547187755713302946

**Discussão**

O módulo aleatório calcula números aleatórios usando o algoritmo Mersenne Twister.

Este é um algoritmo determinístico, mas você pode alterar a semente inicial usando a função `random.seed()`. Por exemplo:

~~~python
random.seed() # Seed based on system time or os.urandom()
random.seed(12345) # Seed based on integer given
random.seed(b'bytedata') # Seed based on byte data
~~~

Além da funcionalidade mostrada, `random()` inclui funções para distribuições uniformes, gaussianas e outras distribuições de probabilidade. Por exemplo, `random.uniform()` calcula números distribuídos uniformemente e `random.gauss()` calcula números distribuídos normalmente. Consulte a documentação para obter informações sobre outras distribuições suportadas.

Funções em `random()` não devem ser usadas em programas relacionados à criptografia. Se você precisar dessa funcionalidade, considere usar funções no módulo ssl. Por exemplo, `ssl.RAND_bytes()` pode ser usado para gerar uma sequência criptograficamente segura de bytes aleatórios.

## 3.12. Conversão de dias para segundos e outras conversões básicas de tempo

**Problema**

Você tem um código que precisa realizar conversões de tempo simples, como dias para segundos, horas para minutos e assim por diante.

**Solução**

Para realizar conversões e aritméticas envolvendo diferentes unidades de tempo, utilize o módulo de data e hora. Por exemplo, para representar um intervalo de tempo, crie uma instância timedelta, assim:

In [150]:
from datetime import timedelta
a = timedelta(days=2, hours=6)
b = timedelta(hours=4.5)
c = a + b
c.days

2

In [151]:
c.seconds

37800

In [152]:
c.seconds / 3600

10.5

In [153]:
c.total_seconds() / 3600

58.5

Se você precisar representar datas e horas específicas, crie instâncias de data e hora e use as operações matemáticas padrão para manipulá-las. Por exemplo:

In [154]:
from datetime import datetime
a = datetime(2012, 9, 23)
print(a + timedelta(days=10))

2012-10-03 00:00:00


In [155]:
b = datetime(2012, 12, 21)
d = b - a
d.days

89

In [156]:
now = datetime.today()
print(now)

2022-10-29 19:38:03.133736


In [157]:
print(now + timedelta(minutes=10))

2022-10-29 19:48:03.133736


Ao fazer cálculos, deve-se notar que `datetime` está ciente dos anos bissextos. Por exemplo:

In [158]:
a = datetime(2012, 3, 1)
b = datetime(2012, 2, 28)
a - b

datetime.timedelta(days=2)

In [159]:
(a - b).days

2

In [160]:
c = datetime(2013, 3, 1)
d = datetime(2013, 2, 28)
(c - d).days

1

**Discussão**

Para a maioria dos problemas básicos de manipulação de data e hora, o módulo datetime será suficiente. Se você precisar realizar manipulações de data mais complexas, como lidar com fusos horários, intervalos de tempo difusos, calcular as datas de feriados e assim por diante, consulte o módulo `dateutil`.

Para ilustrar, muitos cálculos de tempo semelhantes podem ser executados com a função `dateutil.relativedelta()`. No entanto, uma característica notável é que ele preenche algumas lacunas relacionadas ao manuseio de meses (e seu número diferente de dias). Por exemplo:

In [161]:
a = datetime(2012, 9, 23)

~~~python
a + timedelta(months=1)
Traceback (most recent call last):
    File "<stdin>", line 1, in <module>
  TypeError: 'months' is an invalid keyword argument for this function
~~~

In [162]:
from dateutil.relativedelta import relativedelta
a + relativedelta(months=+1)

datetime.datetime(2012, 10, 23, 0, 0)

In [163]:
a + relativedelta(months=+4)

datetime.datetime(2013, 1, 23, 0, 0)

In [164]:
# Time between two dates
b = datetime(2012, 12, 21)
d = b - a
d

datetime.timedelta(days=89)

In [165]:
d = relativedelta(b, a)
d

relativedelta(months=+2, days=+28)

In [166]:
d.months

2

In [167]:
d.days

28

## 3.13. Determinando a data da última sexta-feira

**Problema**

Você quer uma solução geral para encontrar uma data para a última ocorrência de um dia da semana. Sexta-feira passada, por exemplo.

**Solução**

O módulo datetime do Python possui funções e classes utilitárias para ajudar a realizar cálculos como este. Uma solução genérica e decente para esse problema se parece com isso:

In [168]:
from datetime import datetime, timedelta

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday']

def get_previous_byday(dayname, start_date=None):
    if start_date is None:
        start_date = datetime.today()
        day_num = start_date.weekday()
        day_num_target = weekdays.index(dayname)
        days_ago = (7 + day_num - day_num_target) % 7
        if days_ago == 0:
            days_ago = 7
        target_date = start_date - timedelta(days=days_ago)
        return target_date

Usando isso em uma sessão de intérprete ficaria assim:

In [169]:
datetime.today() # For reference

datetime.datetime(2022, 10, 29, 19, 38, 3, 528373)

In [170]:
get_previous_byday('Monday')

datetime.datetime(2022, 10, 24, 19, 38, 3, 569566)

In [171]:
get_previous_byday('Tuesday') # Previous week, not today

datetime.datetime(2022, 10, 25, 19, 38, 3, 601480)

In [172]:
get_previous_byday('Friday')

datetime.datetime(2022, 10, 28, 19, 38, 3, 632398)

O `start_date` opcional pode ser fornecido usando outra instância de `datetime`. Por exemplo:

In [173]:
get_previous_byday('Sunday', datetime(2012, 12, 21))

**Discussão**

Essa receita funciona mapeando a data de início e a data de destino para sua posição numérica na semana (com segunda-feira como dia 0). A aritmética modular é então usada para descobrir há quantos dias a data de destino ocorreu pela última vez. A partir daí, a data desejada é calculada a partir da data de início, subtraindo uma instância de timedelta apropriada.

Se você estiver realizando muitos cálculos de data como este, talvez seja melhor instalar o pacote python-dateutil. Por exemplo, aqui está um exemplo de como realizar o mesmo cálculo usando a função `relativedelta()` do `dateutil`:

In [174]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from dateutil.rrule import *
d = datetime.now()
print(d)

2022-10-29 19:38:03.700004


In [175]:
# Next Friday
print(d + relativedelta(weekday=FR))

2022-11-04 19:38:03.700004


In [176]:
# Last Friday
print(d + relativedelta(weekday=FR(-1)))

2022-10-28 19:38:03.700004


## 3.14. Encontrando o intervalo de datas para o mês atual

**Problema**

Você tem algum código que precisa percorrer cada data no mês atual e deseja uma maneira eficiente de calcular esse intervalo de datas.

**Solução**

Fazer um loop sobre as datas não requer a construção de uma lista de todas as datas com antecedência. Você pode apenas calcular a data inicial e final no intervalo e, em seguida, usar objetos datetime.timedelta para incrementar a data à medida que avança. Aqui está uma função que pega qualquer objeto datetime e retorna uma tupla contendo a primeira data do mês e a data inicial do próximo mês:

In [177]:
from datetime import datetime, date, timedelta
import calendar

def get_month_range(start_date=None):
    if start_date is None:
        start_date = date.today().replace(day=1)
    _,days_in_month = calendar.monthrange(start_date.year, start_date.month)
    end_date = start_date + timedelta(days= days_in_month)
    return (start_date, end_date)   
        

Com isso em vigor, é bem simples fazer um loop no intervalo de datas:

In [178]:
a_day = timedelta(days=1)
first_day, last_day = get_month_range()
while first_day < last_day:
    print(first_day)
    first_day +=a_day

2022-10-01
2022-10-02
2022-10-03
2022-10-04
2022-10-05
2022-10-06
2022-10-07
2022-10-08
2022-10-09
2022-10-10
2022-10-11
2022-10-12
2022-10-13
2022-10-14
2022-10-15
2022-10-16
2022-10-17
2022-10-18
2022-10-19
2022-10-20
2022-10-21
2022-10-22
2022-10-23
2022-10-24
2022-10-25
2022-10-26
2022-10-27
2022-10-28
2022-10-29
2022-10-30
2022-10-31


**Discussão**

Esta receita funciona primeiro calculando um dado correspondente ao primeiro dia do mês. Uma maneira rápida de fazer isso é usar o método **replace()** de um objeto date ou datetime para definir simplesmente ou atribuir dias como **1**. Uma coisa legal sobre o método **replace()** é que ele cria o mesmo tipo de objeto que você iniciou com o método. Assim, se uma entrada for uma instância de dados, o resultado será um dado. Da mesma forma, se a entrada for uma instância de dados e hora, você obterá uma instância de dados e hora.

Depois disso, a função **calendar.monthrange()** é usada para descobrir quantos dias há no mês em questão. Sempre que você precisar obter informações sobre calendários, ou módulo de calendário pode ser útil. **monthrange()** é apenas uma dessas funções que retorna uma tupla contendo o dia da semana junto com o número de dias do mês.

Uma vez que o número de dias do mês é conhecido, os dados finais são compatíveis com os dados iniciais de um tempodelta. É sutil, mas um aspecto importante dessa receita final não deve ser definido no período (na verdade, é o primeiro dia do próximo). Isso espelha o comportamento das operações de fatias e intervalos do Python, que também nunca inclui o ponto final.
Para fazer um loop no intervalo de dados, são usados cálculos matemáticos e de comparação padrão. Por exemplo, as instâncias timedelta podem ser usadas para incrementar dados. O operador **<** é usado para verificar se um dado vem antes da data final.

Idealmente, seria bom criar uma função que funcionasse como a função **range()** integrada, mas para datas. Felizmente, isso é extremamente fácil de implementar usando um gerador:

In [179]:
def date_range(start, stop, step):
    while start < stop:
        yield start
        start +=step

Aqui está um exemplo dele em uso:

In [180]:
for d in date_range(datetime(2012, 9, 1), datetime(2012,10,1), timedelta(hours=6)):
    print(d)

2012-09-01 00:00:00
2012-09-01 06:00:00
2012-09-01 12:00:00
2012-09-01 18:00:00
2012-09-02 00:00:00
2012-09-02 06:00:00
2012-09-02 12:00:00
2012-09-02 18:00:00
2012-09-03 00:00:00
2012-09-03 06:00:00
2012-09-03 12:00:00
2012-09-03 18:00:00
2012-09-04 00:00:00
2012-09-04 06:00:00
2012-09-04 12:00:00
2012-09-04 18:00:00
2012-09-05 00:00:00
2012-09-05 06:00:00
2012-09-05 12:00:00
2012-09-05 18:00:00
2012-09-06 00:00:00
2012-09-06 06:00:00
2012-09-06 12:00:00
2012-09-06 18:00:00
2012-09-07 00:00:00
2012-09-07 06:00:00
2012-09-07 12:00:00
2012-09-07 18:00:00
2012-09-08 00:00:00
2012-09-08 06:00:00
2012-09-08 12:00:00
2012-09-08 18:00:00
2012-09-09 00:00:00
2012-09-09 06:00:00
2012-09-09 12:00:00
2012-09-09 18:00:00
2012-09-10 00:00:00
2012-09-10 06:00:00
2012-09-10 12:00:00
2012-09-10 18:00:00
2012-09-11 00:00:00
2012-09-11 06:00:00
2012-09-11 12:00:00
2012-09-11 18:00:00
2012-09-12 00:00:00
2012-09-12 06:00:00
2012-09-12 12:00:00
2012-09-12 18:00:00
2012-09-13 00:00:00
2012-09-13 06:00:00


Novamente, uma parte importante da facilidade de implementação é que datas e horas podem ser manipuladas usando operadores matemáticos e de comparação padrão.

## 3.15. Convertendo Strings em DateTimes

**Problema**

Seu aplicativo recebe dados temporais em formato de string, mas você deseja converter essas strings em objetos de data e hora para realizar operações não string neles.

**Solução**

O módulo datetime padrão do Python é normalmente a solução fácil para isso. Por exemplo:

In [181]:
from datetime import datetime
text = '2012-09-20'
y = datetime.strptime(text, '%Y-%m-%d')
z = datetime.now()
diff = z - y
diff

datetime.timedelta(days=3691, seconds=70683, microseconds=916057)

**Discussão**
O método `datetime.strptime()` suporta vários códigos de formatação, como `%Y` para o ano de quatro dígitos e `%m` para o mês de dois dígitos. Também vale a pena notar que esses espaços reservados de formatação também funcionam ao contrário, caso você precise representar um objeto de data e hora na saída de string e torná-lo bonito.

Por exemplo, digamos que você tenha algum código que gere um objeto **datetime**, mas você precisa formatar uma data legal e legível para colocar no cabeçalho de uma carta ou relatório gerado automaticamente:

In [182]:
z

datetime.datetime(2022, 10, 29, 19, 38, 3, 916057)

In [183]:
nice_z = datetime.strftime(z, '%A %B %d, %Y')
nice_z

'Saturday October 29, 2022'

Vale a pena notar que o desempenho de `strptime()` geralmente é muito pior do que você poderia esperar, devido ao fato de ser escrito em Python puro e ter que lidar com todos os tipos de configurações de localidade do sistema. Se você estiver analisando muitas datas em seu código e souber o formato preciso, provavelmente obterá um desempenho muito melhor preparando uma solução personalizada. Por exemplo, se você soubesse que as datas estavam no formato **“AAAA-MM-DD”**, você poderia escrever uma função como esta:

In [184]:
from datetime import datetime
def parse_ymd(s):
    year_s,mon_s, day_s = s.split('-')
    return datetime(int(year_s), int(mon_s), int(day_s))

Quando testada, esta função é executada sete vezes mais rápido que `datetime.strptime()`. Isso provavelmente é algo a ser considerado se você estiver processando grandes quantidades de dados envolvendo datas.

## 3.16. Manipulando datas envolvendo fusos horários

**Problema**

Você tinha uma teleconferência agendada para 21 de dezembro de 2012, às 9h30 em Chicago. Em que horário local seu amigo em Bangalore, na Índia, teve que comparecer?

**Solução**

Para quase qualquer problema envolvendo fusos horários, você deve usar o módulo `pytz`. Este pacote fornece o banco de dados de fuso horário Olson, que é o padrão de fato para informações de fuso horário encontradas em muitos idiomas e sistemas operacionais.

Um dos principais usos do `pytz` é localizar datas simples criadas com a biblioteca `datetime`. Por exemplo, veja como você representaria uma data no horário de Chicago:

In [185]:
from datetime import datetime
from pytz import timezone

d = datetime(2012, 12, 21, 9, 30, 0)
print(d)

2012-12-21 09:30:00


In [186]:
# Localize the date for Chicago
central = timezone('US/Central')
loc_d = central.localize(d)
print(loc_d)

2012-12-21 09:30:00-06:00


Uma vez localizada a data, ela pode ser convertida para outros fusos horários. Para encontrar a mesma hora em Bangalore, você faria isso:

In [187]:
# Convert to Bangalore time
bang_d = loc_d.astimezone(timezone('Asia/Kolkata'))
print(bang_d)

2012-12-21 21:00:00+05:30


Se você for realizar aritmética com datas localizadas, precisará estar particularmente atento às transições de horário de verão e outros detalhes. Por exemplo, em 2013, o horário de verão padrão dos EUA começou em 13 de março, às 2h, horário local (nesse ponto, o horário saltou uma hora). Se você estiver executando aritmética ingênua, você errará. Por exemplo:

In [188]:
d = datetime(2013, 3, 10, 1, 45)
loc_d = central.localize(d)
print(loc_d)


2013-03-10 01:45:00-06:00


In [189]:
later = loc_d + timedelta(minutes=30)
print(later)

2013-03-10 02:15:00-06:00


A resposta está errada porque não leva em conta o salto de uma hora na hora local. Para corrigir isso, use o método `normalize()` do fuso horário. Por exemplo:

In [190]:
from datetime import timedelta
later = central.normalize(loc_d + timedelta(minutes=30))
print(later)

2013-03-10 03:15:00-05:00


**Discussão**

Para evitar que sua cabeça exploda completamente, uma estratégia comum para manipulação de data localizada é converter todas as datas para o horário UTC e usá-lo para todo armazenamento interno e manipulação. Por exemplo:

In [191]:
print(loc_d)

2013-03-10 01:45:00-06:00


In [192]:
import pytz
utc_d = loc_d.astimezone(pytz.utc)
print(utc_d)

2013-03-10 07:45:00+00:00


Uma vez no UTC, você não precisa se preocupar com problemas relacionados ao horário de verão e outros assuntos. Assim, você pode simplesmente executar a aritmética de data normal como antes. Se você quiser exibir a data no horário localizado, basta convertê-la para o fuso horário apropriado posteriormente. Por exemplo:

In [193]:
later_utc = utc_d + timedelta(minutes=30)
print(later_utc.astimezone(central))

2013-03-10 03:15:00-05:00


Um problema ao trabalhar com fusos horários é simplesmente descobrir quais nomes de fuso horário usar. Por exemplo, nesta receita, como se sabia que “Asia/Kolkata” era o nome de fuso horário correto para a Índia? Para descobrir, você pode consultar o dicionário `pytz.country_timezones` usando o código do país ISO 3166 como chave. Por exemplo:

In [194]:
pytz.country_timezones['IN']

['Asia/Kolkata']

No momento em que você ler isso, é possível que o módulo `pytz` seja preterido em favor do suporte aprimorado ao fuso horário, conforme descrito no PEP 431. No entanto, muitos dos mesmos problemas ainda serão aplicados (por exemplo, conselhos usando datas UTC etc. ).